---
title: "kaggle exercise 4"
author: "L G H"
date: "2023-05-30"
categories: [data_mining]
---

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

29 58m 44.976s N, 31 8m 3.17625s E
هرم خوفو, شارع ابو الهول السياحي, نزلة البطران, الجيزة, 12125, مصر


In [ ]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 29.97916
Longitude: 31.134215625236113


In [ ]:
universities = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/데이터마이닝/geospatial learn course data/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [ ]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

91.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.759037,-1.252430,POINT (-1.25243 51.75904)
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.562772,7.580947,POINT (7.58095 47.56277)
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179)


In [ ]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [ ]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((180.00000 71.51571, 180.00000 ..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [ ]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,144373535.0,1699876
1,Norway,5347896.0,403336
2,France,67059887.0,2715518
3,Sweden,10285453.0,530883
4,Belarus,9466856.0,63080


In [ ]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((180.00000 71.51571, 180.00000 ...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080


In [ ]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
# print("We located {} universities.".format(len(universities)))
# f-string으로 표현
print(f"We located {len(universities)} universities.")

print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

#print(f"Only {len(universities)} of the universities were located in Europe (in {} different countries).".format(#len(european_universities), len(european_universities.name.unique())))

european_universities.head()

We located 91 universities.
Only 86 of the universities were located in Europe (in 14 different countries).


,Name,Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
0,University of Oxford,51.759037,-1.252430,POINT (-1.25243 51.75904),28,United Kingdom,66834405.0,2829108
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062),28,United Kingdom,66834405.0,2829108
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896),28,United Kingdom,66834405.0,2829108
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179),28,United Kingdom,66834405.0,2829108
5,London School of Economics and Political Science,51.514211,-0.116808,POINT (-0.11681 51.51421),28,United Kingdom,66834405.0,2829108
